In [1]:
from eptr2 import EPTR2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
def convert_to_dataframe(data):
    # Extract the 'items' list, which contains the data entries
    items = data['items']
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(items)
    
    # Optional: Convert date strings to datetime objects if needed
    df['date'] = pd.to_datetime(df['date'])
    
    return df

In [3]:
day_ahead_prices = pd.read_csv("MCP_PREDICTIONS.csv")
day_ahead_prices = day_ahead_prices.iloc[:-24]
day_ahead_prices

,DateTime,renewable_ratio,priceEur,Difference_24,Difference_168,Workday,Workday_168,Workday_24,forecast,BV Forecast,price_lag_24,price_lag_168,forecasted_price_eur
0,2021-01-14T00:00:00Z,0.040737,35.26,1.16,2.17,1,1,1,27.40,4187.97,34.10,33.09,46.060030
1,2021-01-14T01:00:00Z,0.045864,35.13,4.79,0.94,1,1,1,30.10,3975.82,30.34,34.19,44.125388
2,2021-01-14T02:00:00Z,0.051034,34.87,2.32,5.17,1,1,1,37.30,3885.02,32.55,29.70,42.345044
3,2021-01-14T03:00:00Z,0.059953,34.58,7.66,5.35,1,1,1,41.00,3589.79,26.92,29.23,38.267237
4,2021-01-14T04:00:00Z,0.062077,34.57,6.45,7.32,1,1,1,52.60,3137.07,28.12,27.25,37.610058
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28723,2024-04-24T19:00:00Z,0.073455,77.99,0.01,-0.39,1,1,0,98.48,3706.28,77.98,78.38,89.920105
28724,2024-04-24T20:00:00Z,0.073878,77.99,0.01,-0.39,1,1,0,78.06,3386.47,77.98,78.38,89.864853
28725,2024-04-24T21:00:00Z,0.075954,77.99,0.01,1.09,1,1,0,72.15,3207.80,77.98,76.90,88.881698
28726,2024-04-24T22:00:00Z,0.076030,77.99,9.63,20.95,1,1,0,63.76,3123.62,68.36,57.04,74.717507


In [4]:
day_ahead_prices['DateTime'] = pd.to_datetime(day_ahead_prices['DateTime'])
day_ahead_prices = day_ahead_prices.rename(columns={'DateTime': 'date'})

train_day_ahead = day_ahead_prices[day_ahead_prices['date'] < '2023-04-25']
test_day_ahead = day_ahead_prices[day_ahead_prices['date'] >= '2023-04-25']

train_day_ahead['date'] = train_day_ahead['date'].dt.tz_localize(None)
test_day_ahead['date'] = test_day_ahead['date'].dt.tz_localize(None)

train_day_ahead.reset_index(drop=True, inplace=True)
test_day_ahead.reset_index(drop=True, inplace=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\2084121621.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_day_ahead['date'] = train_day_ahead['date'].dt.tz_localize(None)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\2084121621.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_day_ahead['date'] = test_day_ahead['date'].dt.tz_localize(None)


In [5]:
# Find the start and end dates
train_start_date = train_day_ahead['date'].min().strftime('%Y-%m-%d')
train_end_date = train_day_ahead['date'].max().strftime('%Y-%m-%d')

test_start_date = test_day_ahead['date'].min().strftime('%Y-%m-%d')
test_end_date = test_day_ahead['date'].max().strftime('%Y-%m-%d')

print(f"Train start Date: {train_start_date}")
print(f"Train end Date: {train_end_date}")

print(f"Test start Date: {test_start_date}")
print(f"Test end Date: {test_end_date}")

Train start Date: 2021-01-14
Train end Date: 2023-04-24
Test start Date: 2023-04-25
Test end Date: 2024-04-24


In [6]:
train_start_date = pd.to_datetime(train_start_date)
train_end_date = pd.to_datetime(train_end_date)
test_start_date = pd.to_datetime(test_start_date)
test_end_date = pd.to_datetime(test_end_date)

### intra day

In [7]:
def calculate_quantiles(df):
    # Gruplama işlemi
    grouped = df.groupby(['date', 'hour'])
    
    # Sonuçlar için boş liste
    results = []
    
    # Her grup için işlem yapma
    for (date, hour), group in grouped:
        # Fiyata göre sıralama ve kümülatif miktar hesaplama
        group = group.sort_values(by='price')
        group['cumulative_quantity'] = group['quantity'].cumsum()
        
        # Toplam miktarı al
        total_quantity = group['cumulative_quantity'].iloc[-1]
        
        # Yüzdelik dilimler için hedef miktarlar
        target_quantiles = [0.25, 0.5, 0.75]
        targets = [total_quantity * q for q in target_quantiles]
        
        # Hedef miktarlara ulaşan ilk satırları bulma
        for quantile, target in zip(target_quantiles, targets):
            matched_row = group[group['cumulative_quantity'] >= target].iloc[0]
            price_at_quantile = matched_row['price']
            cumulative_lots_at_quantile = total_quantity - matched_row['cumulative_quantity']
            
            # Sonuçları kaydetme
            results.append({
                'date': date,
                'hour': hour,
                'price': price_at_quantile,
                'quantile': quantile,
                'cumulative_lots': cumulative_lots_at_quantile
            })
    
    # Sonuç DataFrame'ini oluşturma ve döndürme
    return pd.DataFrame(results)

In [8]:
eptr = EPTR2(
    ssl_verify=True,
    postprocess=False,  # Set to False to avoid DataFrame conversion
    get_raw_response=False
)

In [9]:
all_intra_day_data = pd.read_csv("all_intraday_data.csv")
all_intra_day_data.head()

,contract_date,contract_hour,contractName,price,quantity,id,prefix,date,hour
0,2021-01-14,00:00,PH21011402,289.98,2,172045281,PH,2021-01-14,02:00
1,2021-01-14,00:00,PH21011402,289.99,20,172045519,PH,2021-01-14,02:00
2,2021-01-14,00:00,PH21011407,269.00,7,172046058,PH,2021-01-14,07:00
3,2021-01-14,00:00,PH21011404,275.00,100,172046087,PH,2021-01-14,04:00
4,2021-01-14,00:00,PH21011408,279.99,20,172045984,PH,2021-01-14,08:00


In [10]:
all_intra_day_data['date'] = pd.to_datetime(all_intra_day_data['date'])

In [11]:
all_intra_day_data = all_intra_day_data.drop('contract_date', axis=1)
all_intra_day_data = all_intra_day_data.drop('contract_hour', axis=1)
all_intra_day_data = all_intra_day_data.drop('id', axis=1)

In [12]:
all_quantiles_data = calculate_quantiles(all_intra_day_data)
print(all_quantiles_data.shape[0])
all_quantiles_data.head(10)

86247


,date,hour,price,quantile,cumulative_lots
0,2021-01-14,02:00,288.99,0.25,1417
1,2021-01-14,02:00,290.00,0.50,0
2,2021-01-14,02:00,290.00,0.75,0
3,2021-01-14,03:00,270.00,0.25,911
4,2021-01-14,03:00,274.98,0.50,618
5,2021-01-14,03:00,278.48,0.75,293
6,2021-01-14,04:00,286.98,0.25,3315
7,2021-01-14,04:00,287.99,0.50,2207
8,2021-01-14,04:00,288.85,0.75,1064
9,2021-01-14,05:00,277.30,0.25,3328


### Production + Day Ahead + Exchange

In [13]:
production_test =pd.read_csv('AlgopolyPerformance-PerformanceReport_2024-04-25_06-16-54.csv')
start_date = str(test_start_date)
end_date = str(test_end_date)

production_test = production_test[(production_test['time'] >= start_date) & (production_test['time'] <= end_date)]
production_test = production_test[(production_test['eic'] == '40W000000010903W')]

production_test = production_test[['time', 'forecast']]

production_test['time'] = pd.to_datetime(production_test['time'])

production_test = production_test.sort_values(by=['time', 'forecast'], ascending=[True, False])

production_test = production_test.drop_duplicates(subset=['time'], keep='first')

production_test = production_test.rename(columns={'forecast': 'predicted_production'})

production_test.rename(columns={'time': 'date'}, inplace=True)

production_test['date'] = production_test['date'].dt.tz_localize(None)

production_test

,date,predicted_production
454702,2023-04-25 00:00:00,5.460000
749147,2023-04-25 01:00:00,7.060000
64952,2023-04-25 02:00:00,7.720000
584674,2023-04-25 03:00:00,9.060000
108499,2023-04-25 04:00:00,14.025820
...,...,...
260510,2024-04-23 19:00:00,51.530000
357861,2024-04-23 20:00:00,53.850000
228946,2024-04-23 21:00:00,58.062974
554018,2024-04-23 22:00:00,53.740000


In [14]:
test_day_ahead

,date,renewable_ratio,priceEur,Difference_24,Difference_168,Workday,Workday_168,Workday_24,forecast,BV Forecast,price_lag_24,price_lag_168,forecasted_price_eur
0,2023-04-25 00:00:00,0.055586,122.57,53.03,10.35,1,1,1,94.57,1821.30,69.54,112.22,99.139640
1,2023-04-25 01:00:00,0.066580,122.57,75.43,15.29,1,1,1,102.25,1506.94,47.14,107.28,84.631126
2,2023-04-25 02:00:00,0.081633,105.39,59.45,12.02,1,1,1,111.28,1324.29,45.94,93.37,75.398830
3,2023-04-25 03:00:00,0.089394,80.14,68.35,-13.23,1,1,1,121.90,1221.89,11.79,93.37,58.079089
4,2023-04-25 04:00:00,0.085833,80.14,44.78,-4.02,1,1,1,134.15,1140.74,35.36,84.16,65.365217
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,2024-04-24 19:00:00,0.073455,77.99,0.01,-0.39,1,1,0,98.48,3706.28,77.98,78.38,89.920105
8780,2024-04-24 20:00:00,0.073878,77.99,0.01,-0.39,1,1,0,78.06,3386.47,77.98,78.38,89.864853
8781,2024-04-24 21:00:00,0.075954,77.99,0.01,1.09,1,1,0,72.15,3207.80,77.98,76.90,88.881698
8782,2024-04-24 22:00:00,0.076030,77.99,9.63,20.95,1,1,0,63.76,3123.62,68.36,57.04,74.717507


In [15]:
test_df = pd.merge(production_test, test_day_ahead[['date','renewable_ratio','Workday','priceEur','forecasted_price_eur']],  on='date', how='left')

In [16]:
test_df

,date,predicted_production,renewable_ratio,Workday,priceEur,forecasted_price_eur
0,2023-04-25 00:00:00,5.460000,0.055586,1,122.57,99.139640
1,2023-04-25 01:00:00,7.060000,0.066580,1,122.57,84.631126
2,2023-04-25 02:00:00,7.720000,0.081633,1,105.39,75.398830
3,2023-04-25 03:00:00,9.060000,0.089394,1,80.14,58.079089
4,2023-04-25 04:00:00,14.025820,0.085833,1,80.14,65.365217
...,...,...,...,...,...,...
8755,2024-04-23 19:00:00,51.530000,0.145035,0,77.98,56.476050
8756,2024-04-23 20:00:00,53.850000,0.152725,0,77.98,55.473172
8757,2024-04-23 21:00:00,58.062974,0.160924,0,77.98,54.403865
8758,2024-04-23 22:00:00,53.740000,0.176247,0,68.36,48.588193


In [17]:
start_date = pd.to_datetime("2023-04-25")
end_date = pd.to_datetime("2024-04-24")
exchange_test = eptr.call("mcp", start_date=start_date, end_date=end_date)
exchange_test = convert_to_dataframe(exchange_test)
exchange_test['exchange_rate'] = exchange_test['price'] / exchange_test['priceEur']
#exchange = pd.concat([exchange]*3, ignore_index=True).sort_values(by='date')
exchange_test['date'] = pd.to_datetime(exchange_test['date'])
exchange_test['date'] = exchange_test['date'].dt.tz_localize(None)
exchange_test

,date,hour,price,priceUsd,priceEur,exchange_rate
0,2023-04-25 00:00:00,00:00,2600.00,134.15,122.57,21.212368
1,2023-04-25 01:00:00,01:00,2600.00,134.15,122.57,21.212368
2,2023-04-25 02:00:00,02:00,2235.58,115.35,105.39,21.212449
3,2023-04-25 03:00:00,03:00,1700.00,87.72,80.14,21.212877
4,2023-04-25 04:00:00,04:00,1700.00,87.72,80.14,21.212877
...,...,...,...,...,...,...
8779,2024-04-24 19:00:00,19:00,2700.00,83.08,77.99,34.619823
8780,2024-04-24 20:00:00,20:00,2700.00,83.08,77.99,34.619823
8781,2024-04-24 21:00:00,21:00,2700.00,83.08,77.99,34.619823
8782,2024-04-24 22:00:00,22:00,2700.00,83.08,77.99,34.619823


In [18]:
test_df = pd.merge(test_df, exchange_test[['date','hour','price','exchange_rate']], on='date', how='left')
test_df

,date,predicted_production,renewable_ratio,Workday,priceEur,forecasted_price_eur,hour,price,exchange_rate
0,2023-04-25 00:00:00,5.460000,0.055586,1,122.57,99.139640,00:00,2600.00,21.212368
1,2023-04-25 01:00:00,7.060000,0.066580,1,122.57,84.631126,01:00,2600.00,21.212368
2,2023-04-25 02:00:00,7.720000,0.081633,1,105.39,75.398830,02:00,2235.58,21.212449
3,2023-04-25 03:00:00,9.060000,0.089394,1,80.14,58.079089,03:00,1700.00,21.212877
4,2023-04-25 04:00:00,14.025820,0.085833,1,80.14,65.365217,04:00,1700.00,21.212877
...,...,...,...,...,...,...,...,...,...
8755,2024-04-23 19:00:00,51.530000,0.145035,0,77.98,56.476050,19:00,2700.00,34.624263
8756,2024-04-23 20:00:00,53.850000,0.152725,0,77.98,55.473172,20:00,2700.00,34.624263
8757,2024-04-23 21:00:00,58.062974,0.160924,0,77.98,54.403865,21:00,2700.00,34.624263
8758,2024-04-23 22:00:00,53.740000,0.176247,0,68.36,48.588193,22:00,2366.89,34.623903


In [19]:
production_train =pd.read_csv('AlgopolyPerformance-PerformanceReport_2024-04-25_06-16-54.csv')
start_date = str(train_start_date)
end_date = str(train_end_date)

production_train = production_train[(production_train['time'] >= start_date) & (production_train['time'] <= end_date)]
production_train = production_train[(production_train['eic'] == '40W000000010903W')]

production_train = production_train[['time', 'forecast']]

production_train['time'] = pd.to_datetime(production_train['time'])

production_train = production_train.sort_values(by=['time', 'forecast'], ascending=[True, False])

production_train = production_train.drop_duplicates(subset=['time'], keep='first')

production_train = production_train.rename(columns={'forecast': 'predicted_production'})

production_train.rename(columns={'time': 'date'}, inplace=True)

production_train['date'] = production_train['date'].dt.tz_localize(None)

production_train

,date,predicted_production
748172,2022-05-23 00:00:00,5.490000
650335,2022-05-23 01:00:00,3.160000
63894,2022-05-23 02:00:00,1.400000
63895,2022-05-23 03:00:00,0.770000
258414,2022-05-23 04:00:00,0.750000
...,...,...
695083,2023-04-23 19:00:00,23.959069
302805,2023-04-23 20:00:00,30.160805
108415,2023-04-23 21:00:00,27.072721
10761,2023-04-23 22:00:00,20.504129


In [20]:
train_day_ahead

,date,renewable_ratio,priceEur,Difference_24,Difference_168,Workday,Workday_168,Workday_24,forecast,BV Forecast,price_lag_24,price_lag_168,forecasted_price_eur
0,2021-01-14 00:00:00,0.040737,35.26,1.16,2.17,1,1,1,27.40,4187.97,34.10,33.09,46.060030
1,2021-01-14 01:00:00,0.045864,35.13,4.79,0.94,1,1,1,30.10,3975.82,30.34,34.19,44.125388
2,2021-01-14 02:00:00,0.051034,34.87,2.32,5.17,1,1,1,37.30,3885.02,32.55,29.70,42.345044
3,2021-01-14 03:00:00,0.059953,34.58,7.66,5.35,1,1,1,41.00,3589.79,26.92,29.23,38.267237
4,2021-01-14 04:00:00,0.062077,34.57,6.45,7.32,1,1,1,52.60,3137.07,28.12,27.25,37.610058
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19939,2023-04-24 19:00:00,0.040206,122.57,43.37,1.00,1,1,0,119.81,3007.49,79.20,121.57,115.630050
19940,2023-04-24 20:00:00,0.036043,122.57,38.66,1.00,1,1,0,117.13,3003.73,83.91,121.57,118.422175
19941,2023-04-24 21:00:00,0.031918,122.57,42.76,1.00,1,1,0,111.04,2930.69,79.81,121.57,117.002163
19942,2023-04-24 22:00:00,0.036577,122.57,66.09,31.20,1,1,0,109.62,2679.69,56.48,91.37,90.715791


In [21]:
train_df = pd.merge(train_day_ahead[['date','renewable_ratio','Workday','priceEur','forecasted_price_eur']],production_train, on='date', how='left')

In [22]:
start_date = pd.to_datetime("2021-01-14")
end_date = pd.to_datetime("2023-04-24")
exchange_train = eptr.call("mcp", start_date=start_date, end_date=end_date)
exchange_train = convert_to_dataframe(exchange_train)
exchange_train['exchange_rate'] = exchange_train['price'] / exchange_train['priceEur']
exchange_train['date'] = pd.to_datetime(exchange_train['date'])
exchange_train['date'] = exchange_train['date'].dt.tz_localize(None)
exchange_train

,date,hour,price,priceUsd,priceEur,exchange_rate
0,2021-01-14 00:00:00,00:00,319.81,42.87,35.26,9.070051
1,2021-01-14 01:00:00,01:00,318.60,42.70,35.13,9.069172
2,2021-01-14 02:00:00,02:00,316.21,42.38,34.87,9.068254
3,2021-01-14 03:00:00,03:00,313.66,42.04,34.58,9.070561
4,2021-01-14 04:00:00,04:00,313.50,42.02,34.57,9.068557
...,...,...,...,...,...,...
19939,2023-04-24 19:00:00,19:00,2600.00,134.15,122.57,21.212368
19940,2023-04-24 20:00:00,20:00,2600.00,134.15,122.57,21.212368
19941,2023-04-24 21:00:00,21:00,2600.00,134.15,122.57,21.212368
19942,2023-04-24 22:00:00,22:00,2600.00,134.15,122.57,21.212368


In [23]:
train_df = pd.merge(train_df, exchange_train[['date','hour','price','exchange_rate']], on='date', how='left')
train_df

,date,renewable_ratio,Workday,priceEur,forecasted_price_eur,predicted_production,hour,price,exchange_rate
0,2021-01-14 00:00:00,0.040737,1,35.26,46.060030,NaN,00:00,319.81,9.070051
1,2021-01-14 01:00:00,0.045864,1,35.13,44.125388,NaN,01:00,318.60,9.069172
2,2021-01-14 02:00:00,0.051034,1,34.87,42.345044,NaN,02:00,316.21,9.068254
3,2021-01-14 03:00:00,0.059953,1,34.58,38.267237,NaN,03:00,313.66,9.070561
4,2021-01-14 04:00:00,0.062077,1,34.57,37.610058,NaN,04:00,313.50,9.068557
...,...,...,...,...,...,...,...,...,...
19939,2023-04-24 19:00:00,0.040206,1,122.57,115.630050,NaN,19:00,2600.00,21.212368
19940,2023-04-24 20:00:00,0.036043,1,122.57,118.422175,NaN,20:00,2600.00,21.212368
19941,2023-04-24 21:00:00,0.031918,1,122.57,117.002163,NaN,21:00,2600.00,21.212368
19942,2023-04-24 22:00:00,0.036577,1,122.57,90.715791,NaN,22:00,2600.00,21.212368


In [24]:
result_df = pd.concat([train_df, test_df], ignore_index=True)
result_df

,date,renewable_ratio,Workday,priceEur,forecasted_price_eur,predicted_production,hour,price,exchange_rate
0,2021-01-14 00:00:00,0.040737,1,35.26,46.060030,NaN,00:00,319.81,9.070051
1,2021-01-14 01:00:00,0.045864,1,35.13,44.125388,NaN,01:00,318.60,9.069172
2,2021-01-14 02:00:00,0.051034,1,34.87,42.345044,NaN,02:00,316.21,9.068254
3,2021-01-14 03:00:00,0.059953,1,34.58,38.267237,NaN,03:00,313.66,9.070561
4,2021-01-14 04:00:00,0.062077,1,34.57,37.610058,NaN,04:00,313.50,9.068557
...,...,...,...,...,...,...,...,...,...
28699,2024-04-23 19:00:00,0.145035,0,77.98,56.476050,51.530000,19:00,2700.00,34.624263
28700,2024-04-23 20:00:00,0.152725,0,77.98,55.473172,53.850000,20:00,2700.00,34.624263
28701,2024-04-23 21:00:00,0.160924,0,77.98,54.403865,58.062974,21:00,2700.00,34.624263
28702,2024-04-23 22:00:00,0.176247,0,68.36,48.588193,53.740000,22:00,2366.89,34.623903


## Convert prices from TL to EUR in all_quantiles_data

In [25]:
# Convert the 'date' column to string format
all_quantiles_data['date'] = all_quantiles_data['date'].astype(str)

# Concatenate the 'date' and 'hour' columns to create a datetime column
all_quantiles_data['date'] = all_quantiles_data['date'] + ' ' + all_quantiles_data['hour']

# Convert the datetime column to datetime format
all_quantiles_data['date'] = pd.to_datetime(all_quantiles_data['date'])
all_quantiles_data

,date,hour,price,quantile,cumulative_lots
0,2021-01-14 02:00:00,02:00,288.99,0.25,1417
1,2021-01-14 02:00:00,02:00,290.00,0.50,0
2,2021-01-14 02:00:00,02:00,290.00,0.75,0
3,2021-01-14 03:00:00,03:00,270.00,0.25,911
4,2021-01-14 03:00:00,03:00,274.98,0.50,618
...,...,...,...,...,...
86242,2024-04-26 22:00:00,22:00,2705.00,0.50,2437
86243,2024-04-26 22:00:00,22:00,2724.00,0.75,1191
86244,2024-04-26 23:00:00,23:00,2700.00,0.25,1948
86245,2024-04-26 23:00:00,23:00,2700.00,0.50,1507


In [26]:
exchange = pd.concat([exchange_train, exchange_test], ignore_index=True)
exchange['date'] = pd.to_datetime(exchange['date'])
exchange

,date,hour,price,priceUsd,priceEur,exchange_rate
0,2021-01-14 00:00:00,00:00,319.81,42.87,35.26,9.070051
1,2021-01-14 01:00:00,01:00,318.60,42.70,35.13,9.069172
2,2021-01-14 02:00:00,02:00,316.21,42.38,34.87,9.068254
3,2021-01-14 03:00:00,03:00,313.66,42.04,34.58,9.070561
4,2021-01-14 04:00:00,04:00,313.50,42.02,34.57,9.068557
...,...,...,...,...,...,...
28723,2024-04-24 19:00:00,19:00,2700.00,83.08,77.99,34.619823
28724,2024-04-24 20:00:00,20:00,2700.00,83.08,77.99,34.619823
28725,2024-04-24 21:00:00,21:00,2700.00,83.08,77.99,34.619823
28726,2024-04-24 22:00:00,22:00,2700.00,83.08,77.99,34.619823


In [27]:
all_quantiles_data = pd.merge(all_quantiles_data, exchange[['date', 'exchange_rate']], on='date', how='left')

In [28]:
all_quantiles_data['priceEUR'] = all_quantiles_data['price'] / all_quantiles_data['exchange_rate']
all_quantiles_data

,date,hour,price,quantile,cumulative_lots,exchange_rate,priceEUR
0,2021-01-14 02:00:00,02:00,288.99,0.25,1417,9.068254,31.868319
1,2021-01-14 02:00:00,02:00,290.00,0.50,0,9.068254,31.979697
2,2021-01-14 02:00:00,02:00,290.00,0.75,0,9.068254,31.979697
3,2021-01-14 03:00:00,03:00,270.00,0.25,911,9.070561,29.766626
4,2021-01-14 03:00:00,03:00,274.98,0.50,618,9.070561,30.315655
...,...,...,...,...,...,...,...
86242,2024-04-26 22:00:00,22:00,2705.00,0.50,2437,NaN,NaN
86243,2024-04-26 22:00:00,22:00,2724.00,0.75,1191,NaN,NaN
86244,2024-04-26 23:00:00,23:00,2700.00,0.25,1948,NaN,NaN
86245,2024-04-26 23:00:00,23:00,2700.00,0.50,1507,NaN,NaN


## Find similar days

In [29]:
import numpy as np

# Initialize an empty list to store similar dates for each row
'''similar_days_list = []

# Iterate over each row in the DataFrame in reverse order
for index, row in result_df[::-1].iterrows():
    # Find the index of previous rows whose priceEur is within the specified range
    similar_rows_index = result_df.loc[:index][(result_df.loc[:index]['priceEur'] >= 0.9 * row['forecasted_price_eur']) &
                                                (result_df.loc[:index]['priceEur'] <= 1.1 * row['forecasted_price_eur']) &
                                                (result_df.loc[:index]['hour'] == row['hour']) &
                                                (result_df.loc[:index]['Workday'] == row['Workday'])].index.tolist()
    # Exclude the current row itself from the similar rows
    similar_rows_index = [idx for idx in similar_rows_index if idx != index]
    
    # Retrieve the dates corresponding to the similar rows' indexes
    similar_dates = result_df.loc[similar_rows_index, 'date'].tolist()
    
    # Extract only the day part from each date and append to the list
    similar_days = [date.date() for date in similar_dates]
    
    # Append the list of similar days to the similar_days_list
    similar_days_list.append(similar_days)

# Reverse the order of lists to match the original DataFrame order
similar_days_list = similar_days_list[::-1]

# Assign the list of similar days to the "similar_dates" column
result_df['similar_days'] = similar_days_list'''


'similar_days_list = []\n\n# Iterate over each row in the DataFrame in reverse order\nfor index, row in result_df[::-1].iterrows():\n    # Find the index of previous rows whose priceEur is within the specified range\n    similar_rows_index = result_df.loc[:index][(result_df.loc[:index][\'priceEur\'] >= 0.9 * row[\'forecasted_price_eur\']) &\n                                                (result_df.loc[:index][\'priceEur\'] <= 1.1 * row[\'forecasted_price_eur\']) &\n                                                (result_df.loc[:index][\'hour\'] == row[\'hour\']) &\n                                                (result_df.loc[:index][\'Workday\'] == row[\'Workday\'])].index.tolist()\n    # Exclude the current row itself from the similar rows\n    similar_rows_index = [idx for idx in similar_rows_index if idx != index]\n    \n    # Retrieve the dates corresponding to the similar rows\' indexes\n    similar_dates = result_df.loc[similar_rows_index, \'date\'].tolist()\n    \n    # Extr

In [30]:
result_df

,date,renewable_ratio,Workday,priceEur,forecasted_price_eur,predicted_production,hour,price,exchange_rate
0,2021-01-14 00:00:00,0.040737,1,35.26,46.060030,NaN,00:00,319.81,9.070051
1,2021-01-14 01:00:00,0.045864,1,35.13,44.125388,NaN,01:00,318.60,9.069172
2,2021-01-14 02:00:00,0.051034,1,34.87,42.345044,NaN,02:00,316.21,9.068254
3,2021-01-14 03:00:00,0.059953,1,34.58,38.267237,NaN,03:00,313.66,9.070561
4,2021-01-14 04:00:00,0.062077,1,34.57,37.610058,NaN,04:00,313.50,9.068557
...,...,...,...,...,...,...,...,...,...
28699,2024-04-23 19:00:00,0.145035,0,77.98,56.476050,51.530000,19:00,2700.00,34.624263
28700,2024-04-23 20:00:00,0.152725,0,77.98,55.473172,53.850000,20:00,2700.00,34.624263
28701,2024-04-23 21:00:00,0.160924,0,77.98,54.403865,58.062974,21:00,2700.00,34.624263
28702,2024-04-23 22:00:00,0.176247,0,68.36,48.588193,53.740000,22:00,2366.89,34.623903


Burada birkaç row için, selected datelerin PTF histogramını çizdir, gerçek PTFsini de nokta olarak koy. 

In [31]:
days_to_solve = result_df[(result_df["date"] >= test_start_date)&(result_df["date"] <= test_end_date)]
days_to_solve

,date,renewable_ratio,Workday,priceEur,forecasted_price_eur,predicted_production,hour,price,exchange_rate
19944,2023-04-25 00:00:00,0.055586,1,122.57,99.139640,5.460000,00:00,2600.00,21.212368
19945,2023-04-25 01:00:00,0.066580,1,122.57,84.631126,7.060000,01:00,2600.00,21.212368
19946,2023-04-25 02:00:00,0.081633,1,105.39,75.398830,7.720000,02:00,2235.58,21.212449
19947,2023-04-25 03:00:00,0.089394,1,80.14,58.079089,9.060000,03:00,1700.00,21.212877
19948,2023-04-25 04:00:00,0.085833,1,80.14,65.365217,14.025820,04:00,1700.00,21.212877
...,...,...,...,...,...,...,...,...,...
28699,2024-04-23 19:00:00,0.145035,0,77.98,56.476050,51.530000,19:00,2700.00,34.624263
28700,2024-04-23 20:00:00,0.152725,0,77.98,55.473172,53.850000,20:00,2700.00,34.624263
28701,2024-04-23 21:00:00,0.160924,0,77.98,54.403865,58.062974,21:00,2700.00,34.624263
28702,2024-04-23 22:00:00,0.176247,0,68.36,48.588193,53.740000,22:00,2366.89,34.623903


In [32]:
unique_days = days_to_solve['date'].dt.date.unique()

In [33]:
unique_days[0]

datetime.date(2023, 4, 25)

### Add KNN Matchings

In [34]:
matchings_df = pd.read_csv("daily_results_df_2.csv")
matchings_df.rename(columns={'Test Date': 'Day'}, inplace=True)
matchings_df.rename(columns={'Prediction Date': 'Matched_Day'}, inplace=True)
matchings_df['Day'] = pd.to_datetime(matchings_df['Day'])
matchings_df['Matched_Day'] = pd.to_datetime(matchings_df['Matched_Day'])
matchings_df = matchings_df.iloc[:-2]

In [35]:
matchings_df

,Day,Matched_Day
0,2023-04-25,2023-04-24
1,2023-04-26,2023-04-25
2,2023-04-27,2023-04-26
3,2023-04-28,2023-04-27
4,2023-04-29,2023-04-28
...,...,...
360,2024-04-19,2023-04-29
361,2024-04-20,2021-12-25
362,2024-04-21,2021-04-07
363,2024-04-22,2023-11-05


In [36]:
all_quantiles_data

,date,hour,price,quantile,cumulative_lots,exchange_rate,priceEUR
0,2021-01-14 02:00:00,02:00,288.99,0.25,1417,9.068254,31.868319
1,2021-01-14 02:00:00,02:00,290.00,0.50,0,9.068254,31.979697
2,2021-01-14 02:00:00,02:00,290.00,0.75,0,9.068254,31.979697
3,2021-01-14 03:00:00,03:00,270.00,0.25,911,9.070561,29.766626
4,2021-01-14 03:00:00,03:00,274.98,0.50,618,9.070561,30.315655
...,...,...,...,...,...,...,...
86242,2024-04-26 22:00:00,22:00,2705.00,0.50,2437,NaN,NaN
86243,2024-04-26 22:00:00,22:00,2724.00,0.75,1191,NaN,NaN
86244,2024-04-26 23:00:00,23:00,2700.00,0.25,1948,NaN,NaN
86245,2024-04-26 23:00:00,23:00,2700.00,0.50,1507,NaN,NaN


In [37]:
import pulp as pl
import pandas as pd

def setup_and_solve_trading_model(all_quantiles_data, production_wind, dayahead):
    """
    Set up and solve the linear programming problem for electricity trading optimization.

    Parameters:
    all_quantiles_data (pd.DataFrame): DataFrame containing hourly quantiles data with 'Hour', 'Quantile', 'Price', and 'Cumulative Lots'.
    production_wind (pd.DataFrame): DataFrame containing production data from wind with columns 'hour' and 'forecast'.
    dayahead (pd.DataFrame): DataFrame containing day-ahead prices with columns 'hour' and 'price'.

    Returns:
    dict: Dictionary containing the solution variables and the objective value.
    """
    # Ensure that dataframes use a consistent 'hour' format, assuming no change needed if already "HH:00"
    # No conversion required for 'dayahead' as it's already "HH:00"

    # Create dictionaries for model parameters
    intraday_dict = all_quantiles_data.set_index(['hour', 'quantile']).to_dict('index')
    production_wind_dict = production_wind.set_index('hour')['forecast'].to_dict()
    dayahead_dict = dayahead.set_index('hour')['price'].to_dict()

    # Set up the model
    model = pl.LpProblem("Electricity_Trading", pl.LpMaximize)

    # Time periods and quantiles
    T = dayahead['hour'].unique()  # Assuming hours are already in 'HH:00' format
    Q = all_quantiles_data['quantile'].unique()

    # Decision Variables
    Q1 = pl.LpVariable.dicts("Q1", T, lowBound=0)
    Q2 = pl.LpVariable.dicts("Q2", [(t, q) for t in T for q in Q], lowBound=0)

    # Objective Function
    model += pl.lpSum([dayahead_dict[t] * Q1[t] for t in T if t in dayahead_dict] +
                      [intraday_dict[(t, q)]['price'] * Q2[(t, q)] for t in T for q in Q if (t, q) in intraday_dict])

    # Constraints
    for t in T:
        if t in production_wind_dict:
            model += Q1[t] + pl.lpSum([Q2[(t, q)] for q in Q if (t, q) in intraday_dict]) <= production_wind_dict[t], f"Production_{t}"

        for q in Q:
            if (t, q) in intraday_dict:
                model += Q2[(t, q)] <= intraday_dict[(t, q)]['cumulative_lots'], f"Demand_{t}_{q}"

    # Solve the model
    model.solve()

    if pl.LpStatus[model.status] != 'Optimal':
        print("No feasible solution found.")
        return None

    # Collect results
    results = {'Variables': {v.name: v.varValue for v in model.variables()},
               'Objective Value': pl.value(model.objective)}

    return results


In [38]:
import pandas as pd
from datetime import datetime, timedelta

daily_results = []

results_list = []

for index, row in matchings_df.iterrows():

    day = row['Day'].date()
    matched_day = row['Matched_Day'].date()

    day_df = days_to_solve[days_to_solve['date'].dt.date == day]
    #day_df = result_df[result_df['date'].dt.date == matched_day]
    day_df['price'].fillna(0, inplace=True)
    day_df['forecasted_price_eur'].fillna(0, inplace=True)
    day_df['exchange_rate'].fillna(0, inplace=True)

    intraday_data = all_quantiles_data[all_quantiles_data["date"].dt.date == matched_day]
    intraday_data['priceEUR'].fillna(0, inplace=True)

    intraday_data = pd.merge(intraday_data[['date','hour','price','quantile','cumulative_lots','priceEUR']], day_df[['hour', 'exchange_rate']], on='hour', how='left')
    
    intraday_data["price"] = intraday_data["exchange_rate"] * intraday_data["priceEUR"]


    # Assuming the structures of production_wind and dayahead for demonstration
    production_wind = pd.DataFrame({'hour': day_df['hour'], 'forecast': day_df['predicted_production']})
    production_wind['forecast'].fillna(0, inplace=True)

    dayahead = pd.DataFrame({'hour': day_df['hour'], 'price': day_df['forecasted_price_eur']*day_df['exchange_rate']})
    
    result = setup_and_solve_trading_model(intraday_data[['hour', 'quantile', 'price', 'cumulative_lots']],
                                                production_wind, dayahead)
    
    # Store result with the date for tracking
    results_list.append({'date': day, 'results': result})
    daily_results.append(result)
    


# Convert results into a DataFrame
results_df = pd.DataFrame(results_list)
results_df

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\197154512.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_df['price'].fillna(0, inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\197154512.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_df['forecasted_price_eur'].fillna(0, inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\197154512.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\197154512.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intraday_data['priceEUR'].fillna(0, inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\197154512.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_df['price'].fillna(0, inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\197154512.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,date,results
0,2023-04-25,"{'Variables': {'Q1_00:00': 5.46, 'Q1_01:00': 7..."
1,2023-04-26,"{'Variables': {'Q1_00:00': 0.0, 'Q1_01:00': 0...."
2,2023-04-27,"{'Variables': {'Q1_00:00': 7.7, 'Q1_01:00': 3...."
3,2023-04-28,"{'Variables': {'Q1_00:00': 0.0, 'Q1_01:00': 0...."
4,2023-04-29,"{'Variables': {'Q1_00:00': 0.0, 'Q1_01:00': 0...."
...,...,...
360,2024-04-19,"{'Variables': {'Q1_00:00': 0.0, 'Q1_01:00': 0...."
361,2024-04-20,"{'Variables': {'Q1_00:00': 0.0, 'Q1_01:00': 0...."
362,2024-04-21,"{'Variables': {'Q1_00:00': 1.08, 'Q1_01:00': 1..."
363,2024-04-22,"{'Variables': {'Q1_00:00': 48.86, 'Q1_01:00': ..."


In [39]:
structured_results = []
current_date = test_start_date

for result in daily_results:
    day_dict = {'Date': current_date.strftime('%Y-%m-%d')}
    day_dict.update(result['Variables'])
    day_dict['Objective Value'] = result['Objective Value']
    structured_results.append(day_dict)
    current_date += timedelta(days=1)

# Create DataFrame from structured results
results_df = pd.DataFrame(structured_results)

# Calculate total revenue
expected_total_revenue = results_df['Objective Value'].sum()

In [40]:
expected_total_revenue

501251657.96316266

In [41]:
#results_df.to_csv('trading_optimization_results_rule_based_expected.csv', index=False)


# KNN CHECK

In [42]:
#results_df = pd.read_csv("trading_optimization_results_expected_knn_model.csv")
results_df

,Date,Q1_00:00,Q1_01:00,Q1_02:00,Q1_03:00,Q1_04:00,Q1_05:00,Q1_06:00,Q1_07:00,Q1_08:00,...,"Q2_('21:00',_0.25)","Q2_('21:00',_0.5)","Q2_('21:00',_0.75)","Q2_('22:00',_0.25)","Q2_('22:00',_0.5)","Q2_('22:00',_0.75)","Q2_('23:00',_0.25)","Q2_('23:00',_0.5)","Q2_('23:00',_0.75)",Objective Value
0,2023-04-25,5.46,7.060000,7.720000,9.060000,14.025820,13.950000,0.000000,0.0,0.000000,...,0.0,0.0,30.650000,0.0,0.0,30.030000,0.0,0.0,30.270000,1.228303e+06
1,2023-04-26,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,16.310000,0.0,0.0,0.000000,0.0,0.0,0.000000,1.638072e+06
2,2023-04-27,7.70,3.200194,5.980000,2.795037,2.360870,3.068171,2.586384,0.0,0.000000,...,0.0,0.0,65.484879,0.0,0.0,65.387576,0.0,0.0,64.222108,1.912169e+06
3,2023-04-28,0.00,0.000000,0.000000,59.798947,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,57.249159,0.0,0.0,0.000000,3.330285e+06
4,2023-04-29,0.00,0.000000,36.585532,33.660453,31.990000,0.000000,36.961611,0.0,0.000000,...,0.0,0.0,35.735778,0.0,0.0,32.514035,0.0,0.0,34.341344,2.277342e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2024-04-19,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,12.590000,0.0,0.0,14.100000,0.0,0.0,15.750000,1.260302e+06
361,2024-04-20,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,1.780000,0.0,0.0,1.460000,0.0,0.0,1.130000,1.084869e+06
362,2024-04-21,1.08,1.210000,1.550000,2.260000,3.000000,3.340000,4.640000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,9.171680e+05
363,2024-04-22,48.86,48.011748,0.000000,0.000000,0.000000,25.140000,21.340000,17.7,14.980000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,6.462460e+05


In [43]:
import pandas as pd

# Assuming results_df is your DataFrame with the provided structure

# Initialize an empty list to store dictionaries
final_data = []

# Iterate over each row in the results_df
for index, row in results_df.iterrows():
    date = row['Date']
    objective_value = row['Objective Value']
    
    # Iterate over each hour column
    for column in results_df.columns:
        if column.startswith('Q1') and column not in ['Date', 'Objective Value']:
            hour = column.split('_')[1]  # Extract hour from column name
            day_ahead_bidded = row[column]
            
            # Append a new dictionary to the final_data list
            final_data.append({'date': date, 'hour': hour, 'day_ahead_bidded': day_ahead_bidded, 'Objective Value': objective_value})

# Convert the list of dictionaries into a DataFrame
final_dayahead_df = pd.DataFrame(final_data)

# Print the final DataFrame
final_dayahead_df


,date,hour,day_ahead_bidded,Objective Value
0,2023-04-25,00:00,5.46000,1.228303e+06
1,2023-04-25,01:00,7.06000,1.228303e+06
2,2023-04-25,02:00,7.72000,1.228303e+06
3,2023-04-25,03:00,9.06000,1.228303e+06
4,2023-04-25,04:00,14.02582,1.228303e+06
...,...,...,...,...
8755,2024-04-23,19:00,0.00000,1.154894e+06
8756,2024-04-23,20:00,0.00000,1.154894e+06
8757,2024-04-23,21:00,0.00000,1.154894e+06
8758,2024-04-23,22:00,0.00000,1.154894e+06


## Real Day Ahead Check

In [44]:
start_date = pd.to_datetime("2023-04-25")
end_date = pd.to_datetime("2024-04-23")
real_dayahead_prices = eptr.call("mcp", start_date=start_date, end_date=end_date)
real_dayahead_prices = convert_to_dataframe(real_dayahead_prices)

In [45]:
real_dayahead_prices = real_dayahead_prices.drop(columns=['priceEur', 'priceUsd'])
real_dayahead_prices['date'] = pd.to_datetime(real_dayahead_prices['date']).dt.tz_localize(None)
real_dayahead_prices

,date,hour,price
0,2023-04-25 00:00:00,00:00,2600.00
1,2023-04-25 01:00:00,01:00,2600.00
2,2023-04-25 02:00:00,02:00,2235.58
3,2023-04-25 03:00:00,03:00,1700.00
4,2023-04-25 04:00:00,04:00,1700.00
...,...,...,...
8755,2024-04-23 19:00:00,19:00,2700.00
8756,2024-04-23 20:00:00,20:00,2700.00
8757,2024-04-23 21:00:00,21:00,2700.00
8758,2024-04-23 22:00:00,22:00,2366.89


In [46]:
final_dayahead_df['date'] = pd.to_datetime(final_dayahead_df['date'] + ' ' + final_dayahead_df['hour'])
real_dayahead_prices['date'] = pd.to_datetime(real_dayahead_prices['date'])

# Merge the DataFrames on the "date" column
dayahead_df = pd.merge(real_dayahead_prices, final_dayahead_df, on="date")
dayahead_df

,date,hour_x,price,hour_y,day_ahead_bidded,Objective Value
0,2023-04-25 00:00:00,00:00,2600.00,00:00,5.46000,1.228303e+06
1,2023-04-25 01:00:00,01:00,2600.00,01:00,7.06000,1.228303e+06
2,2023-04-25 02:00:00,02:00,2235.58,02:00,7.72000,1.228303e+06
3,2023-04-25 03:00:00,03:00,1700.00,03:00,9.06000,1.228303e+06
4,2023-04-25 04:00:00,04:00,1700.00,04:00,14.02582,1.228303e+06
...,...,...,...,...,...,...
8755,2024-04-23 19:00:00,19:00,2700.00,19:00,0.00000,1.154894e+06
8756,2024-04-23 20:00:00,20:00,2700.00,20:00,0.00000,1.154894e+06
8757,2024-04-23 21:00:00,21:00,2700.00,21:00,0.00000,1.154894e+06
8758,2024-04-23 22:00:00,22:00,2366.89,22:00,0.00000,1.154894e+06


In [47]:
dayahead_df['revenue'] = dayahead_df['price'] * dayahead_df['day_ahead_bidded']
dayahead_df

,date,hour_x,price,hour_y,day_ahead_bidded,Objective Value,revenue
0,2023-04-25 00:00:00,00:00,2600.00,00:00,5.46000,1.228303e+06,14196.0000
1,2023-04-25 01:00:00,01:00,2600.00,01:00,7.06000,1.228303e+06,18356.0000
2,2023-04-25 02:00:00,02:00,2235.58,02:00,7.72000,1.228303e+06,17258.6776
3,2023-04-25 03:00:00,03:00,1700.00,03:00,9.06000,1.228303e+06,15402.0000
4,2023-04-25 04:00:00,04:00,1700.00,04:00,14.02582,1.228303e+06,23843.8940
...,...,...,...,...,...,...,...
8755,2024-04-23 19:00:00,19:00,2700.00,19:00,0.00000,1.154894e+06,0.0000
8756,2024-04-23 20:00:00,20:00,2700.00,20:00,0.00000,1.154894e+06,0.0000
8757,2024-04-23 21:00:00,21:00,2700.00,21:00,0.00000,1.154894e+06,0.0000
8758,2024-04-23 22:00:00,22:00,2366.89,22:00,0.00000,1.154894e+06,0.0000


## Real Intraday Check

In [48]:
import pandas as pd

# Assuming results_df is your DataFrame with the provided structure

# Initialize an empty list to store dictionaries
final_data = []

# Iterate over each row in the results_df
for index, row in results_df.iterrows():
    date = row['Date']
    objective_value = row['Objective Value']
    
    # Iterate over each hour column
    for column in results_df.columns:
        if column.startswith('Q2') and column not in ['Date', 'Objective Value']:
            text = column.split('_')
            hour = text[1]  # Extract hour from column name
            quantile = text[-1]
            intraday_bidded = row[column]
            
            # Append a new dictionary to the final_data list
            final_data.append({'date': date, 'hour': hour, 'quantile': quantile, 'intraday_bidded': intraday_bidded, 'Objective Value': objective_value})

# Convert the list of dictionaries into a DataFrame
final_intraday_df = pd.DataFrame(final_data)

# Print the final DataFrame
# Assuming df is your DataFrame

# Remove parentheses and single quotes from the "hour" column
final_intraday_df['hour'] = final_intraday_df['hour'].str.replace("(", "").str.replace(")", "").str.replace("'", "").str.replace(",", "")
final_intraday_df['quantile'] = final_intraday_df['quantile'].str.replace(")", "")

# Print the updated DataFrame
final_intraday_df



,date,hour,quantile,intraday_bidded,Objective Value
0,2023-04-25,00:00,0.25,0.00,1.228303e+06
1,2023-04-25,00:00,0.5,0.00,1.228303e+06
2,2023-04-25,00:00,0.75,0.00,1.228303e+06
3,2023-04-25,01:00,0.25,0.00,1.228303e+06
4,2023-04-25,01:00,0.5,0.00,1.228303e+06
...,...,...,...,...,...
26275,2024-04-23,22:00,0.5,0.00,1.154894e+06
26276,2024-04-23,22:00,0.75,53.74,1.154894e+06
26277,2024-04-23,23:00,0.25,0.00,1.154894e+06
26278,2024-04-23,23:00,0.5,0.00,1.154894e+06


In [49]:
final_intraday_df['date'] = pd.to_datetime(final_intraday_df['date'] + ' ' + final_intraday_df['hour'])
final_intraday_df = final_intraday_df.sort_values(by=['date', 'hour', 'quantile'])
final_intraday_df

,date,hour,quantile,intraday_bidded,Objective Value
0,2023-04-25 00:00:00,00:00,0.25,0.00,1.228303e+06
1,2023-04-25 00:00:00,00:00,0.5,0.00,1.228303e+06
2,2023-04-25 00:00:00,00:00,0.75,0.00,1.228303e+06
3,2023-04-25 01:00:00,01:00,0.25,0.00,1.228303e+06
4,2023-04-25 01:00:00,01:00,0.5,0.00,1.228303e+06
...,...,...,...,...,...
26275,2024-04-23 22:00:00,22:00,0.5,0.00,1.154894e+06
26276,2024-04-23 22:00:00,22:00,0.75,53.74,1.154894e+06
26277,2024-04-23 23:00:00,23:00,0.25,0.00,1.154894e+06
26278,2024-04-23 23:00:00,23:00,0.5,0.00,1.154894e+06


In [50]:
import pandas as pd

# Create an empty DataFrame
real_intra_day_data = pd.DataFrame(columns=['date', 'hour', 'price', 'quantile', 'cumulative_lots', 'priceEUR', 'exchange_rate'])

# Your for loop
for index, row in matchings_df.iterrows():
    day = row['Day'].date()
    matched_day = row['Matched_Day'].date()

    day_df = days_to_solve[days_to_solve['date'].dt.date == day]
    day_df['price'].fillna(0, inplace=True)
    day_df['forecasted_price_eur'].fillna(0, inplace=True)
    day_df['exchange_rate'].fillna(0, inplace=True)

    intraday_data = all_quantiles_data[all_quantiles_data["date"].dt.date == matched_day]
    intraday_data['priceEUR'].fillna(0, inplace=True)

    intraday_data = pd.merge(intraday_data[['date','hour','price','quantile','cumulative_lots','priceEUR']], day_df[['hour', 'exchange_rate']], on='hour', how='left')
    
    intraday_data["price"] = intraday_data["exchange_rate"] * intraday_data["priceEUR"]
    intraday_data["date"] = day
    
    # Concatenate rows with result_df
    real_intra_day_data = pd.concat([real_intra_day_data, intraday_data])

# Print the result DataFrame
real_intra_day_data


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\1563669757.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_df['price'].fillna(0, inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\1563669757.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_df['forecasted_price_eur'].fillna(0, inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\1563669757.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

,date,hour,price,quantile,cumulative_lots,priceEUR,exchange_rate
0,2023-04-25,00:00,1500.119934,0.25,5096,70.719116,21.212368
1,2023-04-25,00:00,1575.125431,0.50,3407,74.255048,21.212368
2,2023-04-25,00:00,1635.329843,0.75,1704,77.093223,21.212368
3,2023-04-25,01:00,999.941049,0.25,6947,47.139529,21.212368
4,2023-04-25,01:00,1079.446362,0.50,4755,50.887593,21.212368
...,...,...,...,...,...,...,...
67,2024-04-23,22:00,1936.289992,0.50,5520,55.923505,34.623903
68,2024-04-23,22:00,1985.647000,0.75,2665,57.349023,34.623903
69,2024-04-23,23:00,1579.797721,0.25,11783,45.626491,34.624572
70,2024-04-23,23:00,1633.923553,0.50,7809,47.189711,34.624572


In [51]:
real_intra_day_data = real_intra_day_data.dropna()
real_intra_day_data

,date,hour,price,quantile,cumulative_lots,priceEUR,exchange_rate
0,2023-04-25,00:00,1500.119934,0.25,5096,70.719116,21.212368
1,2023-04-25,00:00,1575.125431,0.50,3407,74.255048,21.212368
2,2023-04-25,00:00,1635.329843,0.75,1704,77.093223,21.212368
3,2023-04-25,01:00,999.941049,0.25,6947,47.139529,21.212368
4,2023-04-25,01:00,1079.446362,0.50,4755,50.887593,21.212368
...,...,...,...,...,...,...,...
67,2024-04-23,22:00,1936.289992,0.50,5520,55.923505,34.623903
68,2024-04-23,22:00,1985.647000,0.75,2665,57.349023,34.623903
69,2024-04-23,23:00,1579.797721,0.25,11783,45.626491,34.624572
70,2024-04-23,23:00,1633.923553,0.50,7809,47.189711,34.624572


In [52]:
real_intra_day_data['date'] = real_intra_day_data['date'].astype(str)

real_intra_day_data['date'] = pd.to_datetime(real_intra_day_data['date'] + ' ' + real_intra_day_data['hour'])

real_intra_day_data['date'] = pd.to_datetime(real_intra_day_data['date'])

real_intra_day_data

,date,hour,price,quantile,cumulative_lots,priceEUR,exchange_rate
0,2023-04-25 00:00:00,00:00,1500.119934,0.25,5096,70.719116,21.212368
1,2023-04-25 00:00:00,00:00,1575.125431,0.50,3407,74.255048,21.212368
2,2023-04-25 00:00:00,00:00,1635.329843,0.75,1704,77.093223,21.212368
3,2023-04-25 01:00:00,01:00,999.941049,0.25,6947,47.139529,21.212368
4,2023-04-25 01:00:00,01:00,1079.446362,0.50,4755,50.887593,21.212368
...,...,...,...,...,...,...,...
67,2024-04-23 22:00:00,22:00,1936.289992,0.50,5520,55.923505,34.623903
68,2024-04-23 22:00:00,22:00,1985.647000,0.75,2665,57.349023,34.623903
69,2024-04-23 23:00:00,23:00,1579.797721,0.25,11783,45.626491,34.624572
70,2024-04-23 23:00:00,23:00,1633.923553,0.50,7809,47.189711,34.624572


In [53]:
real_intra_day_data['quantile'] = real_intra_day_data['quantile'].astype(float)
final_intraday_df['quantile'] = final_intraday_df['quantile'].astype(float)

In [54]:
# Assuming you have imported pandas as pd

# Perform outer join
final_intraday_df = pd.merge(real_intra_day_data, final_intraday_df, on=['date', 'hour', 'quantile'], how='outer')
final_intraday_df

,date,hour,price,quantile,cumulative_lots,priceEUR,exchange_rate,intraday_bidded,Objective Value
0,2023-04-25 00:00:00,00:00,1500.119934,0.25,5096,70.719116,21.212368,0.0,1.228303e+06
1,2023-04-25 00:00:00,00:00,1575.125431,0.50,3407,74.255048,21.212368,0.0,1.228303e+06
2,2023-04-25 00:00:00,00:00,1635.329843,0.75,1704,77.093223,21.212368,0.0,1.228303e+06
3,2023-04-25 01:00:00,01:00,999.941049,0.25,6947,47.139529,21.212368,0.0,1.228303e+06
4,2023-04-25 01:00:00,01:00,1079.446362,0.50,4755,50.887593,21.212368,0.0,1.228303e+06
...,...,...,...,...,...,...,...,...,...
26275,2024-04-04 20:00:00,20:00,NaN,0.50,NaN,NaN,NaN,NaN,2.314489e+05
26276,2024-04-04 20:00:00,20:00,NaN,0.75,NaN,NaN,NaN,NaN,2.314489e+05
26277,2024-04-04 21:00:00,21:00,NaN,0.25,NaN,NaN,NaN,NaN,2.314489e+05
26278,2024-04-04 21:00:00,21:00,NaN,0.50,NaN,NaN,NaN,NaN,2.314489e+05


In [55]:
final_intraday_df = final_intraday_df.dropna()
final_intraday_df

,date,hour,price,quantile,cumulative_lots,priceEUR,exchange_rate,intraday_bidded,Objective Value
0,2023-04-25 00:00:00,00:00,1500.119934,0.25,5096,70.719116,21.212368,0.00,1.228303e+06
1,2023-04-25 00:00:00,00:00,1575.125431,0.50,3407,74.255048,21.212368,0.00,1.228303e+06
2,2023-04-25 00:00:00,00:00,1635.329843,0.75,1704,77.093223,21.212368,0.00,1.228303e+06
3,2023-04-25 01:00:00,01:00,999.941049,0.25,6947,47.139529,21.212368,0.00,1.228303e+06
4,2023-04-25 01:00:00,01:00,1079.446362,0.50,4755,50.887593,21.212368,0.00,1.228303e+06
...,...,...,...,...,...,...,...,...,...
26257,2024-04-23 22:00:00,22:00,1936.289992,0.50,5520,55.923505,34.623903,0.00,1.154894e+06
26258,2024-04-23 22:00:00,22:00,1985.647000,0.75,2665,57.349023,34.623903,53.74,1.154894e+06
26259,2024-04-23 23:00:00,23:00,1579.797721,0.25,11783,45.626491,34.624572,0.00,1.154894e+06
26260,2024-04-23 23:00:00,23:00,1633.923553,0.50,7809,47.189711,34.624572,0.00,1.154894e+06


In [56]:
final_intraday_df = final_intraday_df.sort_values(by=['date', 'hour', 'quantile'])
final_intraday_df

,date,hour,price,quantile,cumulative_lots,priceEUR,exchange_rate,intraday_bidded,Objective Value
0,2023-04-25 00:00:00,00:00,1500.119934,0.25,5096,70.719116,21.212368,0.00,1.228303e+06
1,2023-04-25 00:00:00,00:00,1575.125431,0.50,3407,74.255048,21.212368,0.00,1.228303e+06
2,2023-04-25 00:00:00,00:00,1635.329843,0.75,1704,77.093223,21.212368,0.00,1.228303e+06
3,2023-04-25 01:00:00,01:00,999.941049,0.25,6947,47.139529,21.212368,0.00,1.228303e+06
4,2023-04-25 01:00:00,01:00,1079.446362,0.50,4755,50.887593,21.212368,0.00,1.228303e+06
...,...,...,...,...,...,...,...,...,...
26257,2024-04-23 22:00:00,22:00,1936.289992,0.50,5520,55.923505,34.623903,0.00,1.154894e+06
26258,2024-04-23 22:00:00,22:00,1985.647000,0.75,2665,57.349023,34.623903,53.74,1.154894e+06
26259,2024-04-23 23:00:00,23:00,1579.797721,0.25,11783,45.626491,34.624572,0.00,1.154894e+06
26260,2024-04-23 23:00:00,23:00,1633.923553,0.50,7809,47.189711,34.624572,0.00,1.154894e+06


In [57]:
final_intraday_df = final_intraday_df[["date","hour","price","quantile","intraday_bidded"]]
final_intraday_df

,date,hour,price,quantile,intraday_bidded
0,2023-04-25 00:00:00,00:00,1500.119934,0.25,0.00
1,2023-04-25 00:00:00,00:00,1575.125431,0.50,0.00
2,2023-04-25 00:00:00,00:00,1635.329843,0.75,0.00
3,2023-04-25 01:00:00,01:00,999.941049,0.25,0.00
4,2023-04-25 01:00:00,01:00,1079.446362,0.50,0.00
...,...,...,...,...,...
26257,2024-04-23 22:00:00,22:00,1936.289992,0.50,0.00
26258,2024-04-23 22:00:00,22:00,1985.647000,0.75,53.74
26259,2024-04-23 23:00:00,23:00,1579.797721,0.25,0.00
26260,2024-04-23 23:00:00,23:00,1633.923553,0.50,0.00


In [58]:
all_intra_day_data = pd.read_csv("all_intraday_data.csv")
all_intra_day_data = all_intra_day_data.drop('contract_date', axis=1)
all_intra_day_data = all_intra_day_data.drop('contract_hour', axis=1)
all_intra_day_data = all_intra_day_data.drop('contractName', axis=1)
all_intra_day_data = all_intra_day_data.drop('prefix', axis=1)
all_intra_day_data = all_intra_day_data.drop('id', axis=1)
all_intra_day_data = all_intra_day_data[(all_intra_day_data["date"] >= "2023-04-25") & (all_intra_day_data["date"] <= "2024-04-23")]

all_intra_day_data['date'] = all_intra_day_data['date'].astype(str)
all_intra_day_data['date'] = pd.to_datetime(all_intra_day_data['date'] + ' ' + all_intra_day_data['hour'])
all_intra_day_data['date'] = pd.to_datetime(all_intra_day_data['date'])

all_intra_day_data.reset_index(drop=True, inplace=True)


all_intra_day_data

,price,quantity,date,hour
0,2522.04,150,2023-04-25 00:00:00,00:00
1,2525.01,150,2023-04-25 00:00:00,00:00
2,2600.00,3,2023-04-25 00:00:00,00:00
3,2525.02,11,2023-04-25 00:00:00,00:00
4,2525.03,5,2023-04-25 00:00:00,00:00
...,...,...,...,...
3902846,2649.00,5,2024-04-23 23:00:00,23:00
3902847,2649.00,1,2024-04-23 23:00:00,23:00
3902848,2650.00,24,2024-04-23 23:00:00,23:00
3902849,2650.00,25,2024-04-23 23:00:00,23:00


In [59]:
all_intra_day_data = all_intra_day_data.sort_values(by=['date', 'hour', 'price'])
all_intra_day_data

,price,quantity,date,hour
1894,2302.02,9,2023-04-25 00:00:00,00:00
1895,2302.02,2,2023-04-25 00:00:00,00:00
1893,2302.03,1,2023-04-25 00:00:00,00:00
1858,2357.03,2,2023-04-25 00:00:00,00:00
1862,2359.68,2,2023-04-25 00:00:00,00:00
...,...,...,...,...
3902827,2714.90,9,2024-04-23 23:00:00,23:00
3902829,2714.90,9,2024-04-23 23:00:00,23:00
3902830,2714.90,4,2024-04-23 23:00:00,23:00
3902828,2715.00,2,2024-04-23 23:00:00,23:00


In [60]:
for index, row in final_intraday_df.iterrows():
    if row["intraday_bidded"] == 0:
        final_intraday_df.at[index, "revenue"] = 0

    else:
        date = row["date"]
        price = row["price"]
        quantity = row["intraday_bidded"]
        threshold_quantity = all_intra_day_data[(all_intra_day_data["date"] == date) & (all_intra_day_data["price"] >= price)]["quantity"].sum()

        if threshold_quantity is None:
            final_intraday_df.at[index, "revenue"] = 0
        else:
            final_intraday_df.at[index, "revenue"] = min(threshold_quantity, quantity) * price
        

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\2743370772.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_intraday_df.at[index, "revenue"] = 0


In [61]:
final_intraday_df

,date,hour,price,quantile,intraday_bidded,revenue
0,2023-04-25 00:00:00,00:00,1500.119934,0.25,0.00,0.000000
1,2023-04-25 00:00:00,00:00,1575.125431,0.50,0.00,0.000000
2,2023-04-25 00:00:00,00:00,1635.329843,0.75,0.00,0.000000
3,2023-04-25 01:00:00,01:00,999.941049,0.25,0.00,0.000000
4,2023-04-25 01:00:00,01:00,1079.446362,0.50,0.00,0.000000
...,...,...,...,...,...,...
26257,2024-04-23 22:00:00,22:00,1936.289992,0.50,0.00,0.000000
26258,2024-04-23 22:00:00,22:00,1985.647000,0.75,53.74,106708.669785
26259,2024-04-23 23:00:00,23:00,1579.797721,0.25,0.00,0.000000
26260,2024-04-23 23:00:00,23:00,1633.923553,0.50,0.00,0.000000


In [62]:
intraday_bids = final_intraday_df["intraday_bidded"].sum()
intraday_bids

153831.27168930203

In [63]:
dayahead_df

,date,hour_x,price,hour_y,day_ahead_bidded,Objective Value,revenue
0,2023-04-25 00:00:00,00:00,2600.00,00:00,5.46000,1.228303e+06,14196.0000
1,2023-04-25 01:00:00,01:00,2600.00,01:00,7.06000,1.228303e+06,18356.0000
2,2023-04-25 02:00:00,02:00,2235.58,02:00,7.72000,1.228303e+06,17258.6776
3,2023-04-25 03:00:00,03:00,1700.00,03:00,9.06000,1.228303e+06,15402.0000
4,2023-04-25 04:00:00,04:00,1700.00,04:00,14.02582,1.228303e+06,23843.8940
...,...,...,...,...,...,...,...
8755,2024-04-23 19:00:00,19:00,2700.00,19:00,0.00000,1.154894e+06,0.0000
8756,2024-04-23 20:00:00,20:00,2700.00,20:00,0.00000,1.154894e+06,0.0000
8757,2024-04-23 21:00:00,21:00,2700.00,21:00,0.00000,1.154894e+06,0.0000
8758,2024-04-23 22:00:00,22:00,2366.89,22:00,0.00000,1.154894e+06,0.0000


In [64]:
dayahead_bids = dayahead_df["day_ahead_bidded"].sum()
dayahead_bids

81524.60510511269

In [65]:
real_revenue = final_intraday_df['revenue'].sum() + dayahead_df['revenue'].sum()

print("Expected revenue: ", expected_total_revenue)
print("Real revenue: ", real_revenue)


Expected revenue:  501251657.96316266
Real revenue:  288529660.69628286


In [66]:
dayahead_df['date'] = pd.to_datetime(dayahead_df['date'])

# Extract month from 'date' column
dayahead_df['month'] = dayahead_df['date'].dt.month

# Group by month and calculate average revenue
dayahead_monthly_total_revenue = dayahead_df.groupby('month')['revenue'].sum()

print("Monthly total revenue from dayahead: ", dayahead_monthly_total_revenue)
print("Monthly avg revenue from dayahead: ", dayahead_monthly_total_revenue.mean())
print("Total yearly revenue from dayahead: ", dayahead_monthly_total_revenue.sum())

Monthly total revenue from dayahead:  month
1     1.793969e+07
2     1.418860e+07
3     1.252531e+07
4     9.869584e+06
5     1.283754e+07
6     1.368826e+07
7     2.118711e+07
8     1.740438e+07
9     1.827571e+07
10    1.015923e+07
11    9.372269e+06
12    1.025945e+07
Name: revenue, dtype: float64
Monthly avg revenue from dayahead:  13975593.699608393
Total yearly revenue from dayahead:  167707124.39530072


In [67]:
final_intraday_df['date'] = pd.to_datetime(final_intraday_df['date'])

# Extract month from 'date' column
final_intraday_df['month'] = final_intraday_df['date'].dt.month

# Group by month and calculate average revenue
intraday_monthly_total_revenue = final_intraday_df.groupby('month')['revenue'].sum()

print("Monthly total revenue from intraday: ", intraday_monthly_total_revenue)
print("Monthly avg revenue from intraday: ", intraday_monthly_total_revenue.mean())
print("Total yearly revenue from intraday: ", intraday_monthly_total_revenue.sum())


Monthly total revenue from intraday:  month
1     1.553262e+07
2     6.629511e+06
3     3.100282e+06
4     1.131110e+07
5     8.006924e+06
6     4.829088e+06
7     1.291425e+07
8     1.482237e+07
9     1.532311e+07
10    1.062238e+07
11    1.039972e+07
12    7.331180e+06
Name: revenue, dtype: float64
Monthly avg revenue from intraday:  10068544.691748511
Total yearly revenue from intraday:  120822536.30098213


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\3558484632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_intraday_df['date'] = pd.to_datetime(final_intraday_df['date'])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\3558484632.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_intraday_df['month'] = final_intraday_df['date'].dt.month


In [68]:
total_monthly_revenue = dayahead_monthly_total_revenue + intraday_monthly_total_revenue
print("Total revenue: ", total_monthly_revenue.sum())

Total revenue:  288529660.69628286


In [69]:
# Convert the 'date' column to datetime if not already done
dayahead_df['date'] = pd.to_datetime(dayahead_df['date'])

# Extract month and month name from 'date' column
dayahead_df['month'] = dayahead_df['date'].dt.month
dayahead_df['month_name'] = dayahead_df['date'].dt.strftime('%B')  # %B gives full month name

# Group by month and calculate total revenue
dayahead_monthly_total_revenue = dayahead_df.groupby(['month', 'month_name'])['revenue'].sum()

# Print monthly revenues with month names
print("Monthly revenues from dayahead:")
for month, revenue in dayahead_monthly_total_revenue.items():
    print(f"{month[1]}: {revenue}")

print("Monthly avg revenue from dayahead:", dayahead_monthly_total_revenue.mean())
print("Total yearly revenue from dayahead:", dayahead_monthly_total_revenue.sum())


Monthly revenues from dayahead:
January: 17939686.88298495
February: 14188596.894075997
March: 12525314.944857616
April: 9869583.796039904
May: 12837536.563241024
June: 13688259.590573937
July: 21187108.896156203
August: 17404384.667676836
September: 18275709.787606694
October: 10159226.648447398
November: 9372269.443984363
December: 10259446.27965582
Monthly avg revenue from dayahead: 13975593.699608393
Total yearly revenue from dayahead: 167707124.39530072


In [70]:
# Convert the 'date' column to datetime if not already done
final_intraday_df['date'] = pd.to_datetime(final_intraday_df['date'])

# Extract month and month name from 'date' column
final_intraday_df['month'] = final_intraday_df['date'].dt.month
final_intraday_df['month_name'] = final_intraday_df['date'].dt.strftime('%B')  # %B gives full month name

# Group by month and calculate total revenue
intraday_monthly_total_revenue = final_intraday_df.groupby(['month', 'month_name'])['revenue'].sum()

# Print monthly revenues with month names
print("Monthly revenues from intraday:")
for month, revenue in intraday_monthly_total_revenue.items():
    print(f"{month[1]}: {revenue}")

print("Monthly avg revenue from intraday:", intraday_monthly_total_revenue.mean())
print("Total yearly revenue from intraday:", intraday_monthly_total_revenue.sum())


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\1491771407.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_intraday_df['date'] = pd.to_datetime(final_intraday_df['date'])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\1491771407.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_intraday_df['month'] = final_intraday_df['date'].dt.month


Monthly revenues from intraday:
January: 15532622.579601998
February: 6629511.203484942
March: 3100281.6362845455
April: 11311099.19408213
May: 8006923.712372678
June: 4829087.755607602
July: 12914248.980260227
August: 14822367.564113181
September: 15323112.965436602
October: 10622380.752553863
November: 10399719.873647794
December: 7331180.083536561
Monthly avg revenue from intraday: 10068544.691748511
Total yearly revenue from intraday: 120822536.30098213


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19096\1491771407.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_intraday_df['month_name'] = final_intraday_df['date'].dt.strftime('%B')  # %B gives full month name
